In [247]:
batch_size = 1
look_back = 5

In [241]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

def create_klines_dataset(data) :
    X_train = []
    y_train = []

    for i in range(look_back, len(data)):
        X_train.append(data[i-look_back:i, 0:9])
        y_train.append(data[i, 3])

    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], look_back, 9))

    return X_train, y_train

import pandas as pd

# klines_dataset = pd.read_csv('resource/dataset/binance/klines-BTCUSDT-1m_20220101090000_20220101205900.csv')
klines_dataset = pd.read_csv('resource/dataset/binance/klines-BTCUSDT-1m_20220101210000_20220102085900.csv')
del klines_dataset['openTime']
del klines_dataset['closeTime']
del klines_dataset['ignore']

scaler = MinMaxScaler(feature_range=(0, 1))
klines_dataset_scaled = scaler.fit_transform(klines_dataset)
# klines_dataset
# klines_dataset_scaled.shape
x_train,y_train = create_klines_dataset(klines_dataset_scaled)




In [252]:
y_train[0]

0.008537321072736859

In [255]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

def create_klines_dataset(data) :
    y = np.array(data[4])
    del data['openTime']
    del data['closeTime']
    # del data['ignore']
    x = np.array(data)

    x = np.reshape(x, (x.shape[0],  look_back, 1))
    y = np.reshape(y, (y.shape[0],  look_back, 1))

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=1)

    return x_train, x_val, x_test, y_train, y_val, y_test

import pandas as pd

# klines_dataset = pd.read_csv('resource/dataset/binance/klines-BTCUSDT-1m_20220101090000_20220101205900.csv')
klines_dataset = pd.read_csv('resource/dataset/binance/klines-BTCUSDT-1m_20220101210000_20220102085900.csv')
del klines_dataset['openTime']
del klines_dataset['closeTime']
del klines_dataset['ignore']

scaler = MinMaxScaler(feature_range=(0, 1))
klines_dataset.shape
# klines_dataset_scaled = scaler.fit_transform(klines_dataset)
# # klines_dataset_scaled.shape
# x_train, x_val, x_test, y_train, y_val, y_test = create_klines_dataset(klines_dataset_scaled)

(720, 9)

In [201]:
klines_dataset

,open,high,low,close,volume,quoteAssetVolume,numberOfTrades,takerBuyBaseAssetVolume,takerBuyQuoteAssetVolume
0,46758.87,46826.83,46756.30,46825.01,7.61286,3.562220e+05,501,3.79922,1.777819e+05
1,46825.01,46825.02,46800.49,46807.99,7.62883,3.570913e+05,381,2.99694,1.402743e+05
2,46805.14,46814.42,46761.76,46808.04,33.52525,1.568461e+06,781,18.85000,8.817704e+05
3,46808.04,46815.26,46792.36,46793.94,4.82469,2.258093e+05,437,0.92098,4.310377e+04
4,46793.94,46826.00,46793.94,46798.81,5.88667,2.755360e+05,430,3.03827,1.422078e+05
...,...,...,...,...,...,...,...,...,...
715,47687.26,47778.48,47607.26,47734.33,87.40123,4.168367e+06,1359,46.90157,2.237324e+06
716,47728.96,47748.81,47699.54,47739.99,19.58113,9.343993e+05,650,7.49365,3.576146e+05
717,47740.00,47799.88,47711.43,47771.53,32.42206,1.548339e+06,904,15.75298,7.522625e+05
718,47771.52,47771.53,47720.25,47722.99,20.97965,1.001710e+06,618,5.00430,2.389313e+05


In [105]:
x_train.shape

(350, 10, 1)

In [248]:
import tensorflow as tf

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(32, batch_input_shape=(batch_size, look_back, 9), stateful=True, return_sequences=True))
model.add(tf.keras.layers.Dropout(0.2)) # overfitting을 막기 위해 20% 가량을 drop
model.add(tf.keras.layers.LSTM(32, batch_input_shape=(batch_size, look_back, 9), stateful=True))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1))
# model.add(tf.keras.layers.Dense(64, input_dim= 8, activation='relu'))
# model.add(tf.keras.layers.Dense(8, activation='relu'))
# model.add(tf.keras.layers.Dense(1))

In [249]:
# sgd = tf.keras.optimizers.Adam(learning_rate=0.00000001)
# model.compile(loss='mse',optimizer=sgd,metrics=['accuracy'])
model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])

In [183]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_19 (LSTM)              (5, 3, 32)                5376      
                                                                 
 dropout_18 (Dropout)        (5, 3, 32)                0         
                                                                 
 lstm_20 (LSTM)              (5, 32)                   8320      
                                                                 
 dropout_19 (Dropout)        (5, 32)                   0         
                                                                 
 dense_12 (Dense)            (5, 1)                    33        
                                                                 
Total params: 13,729
Trainable params: 13,729
Non-trainable params: 0
_________________________________________________________________


In [251]:
hist = model.fit(x_train, y_train, epochs=50, batch_size=batch_size, verbose=1)

Epoch 1/50
715/715 [==============================] - 2s 2ms/step - loss: 0.0033 - accuracy: 0.0014
Epoch 2/50
715/715 [==============================] - 2s 2ms/step - loss: 0.0028 - accuracy: 0.0014
Epoch 3/50
715/715 [==============================] - 2s 2ms/step - loss: 0.0027 - accuracy: 0.0014
Epoch 4/50
715/715 [==============================] - 2s 2ms/step - loss: 0.0025 - accuracy: 0.0014
Epoch 5/50
715/715 [==============================] - 2s 2ms/step - loss: 0.0024 - accuracy: 0.0014
Epoch 6/50
715/715 [==============================] - 2s 2ms/step - loss: 0.0025 - accuracy: 0.0014
Epoch 7/50
715/715 [==============================] - 2s 2ms/step - loss: 0.0021 - accuracy: 0.0014
Epoch 8/50
715/715 [==============================] - 2s 2ms/step - loss: 0.0022 - accuracy: 0.0014
Epoch 9/50
715/715 [==============================] - 2s 2ms/step - loss: 0.0021 - accuracy: 0.0014
Epoch 10/50
715/715 [==============================] - 2s 2ms/step - loss: 0.0019 - accuracy: 0.0014


KeyboardInterrupt



In [253]:
y_predict = model.predict(x_train, batch_size)
y_predict

array([[-3.59565020e-04],
       [ 5.54576516e-03],
       [ 3.30649018e-02],
       [ 6.69213235e-02],
       [ 7.87526667e-02],
       [ 1.07704818e-01],
       [ 1.11235231e-01],
       [ 9.21479464e-02],
       [ 9.12381411e-02],
       [ 1.01954579e-01],
       [ 1.47098303e-01],
       [ 1.60551280e-01],
       [ 1.41936481e-01],
       [ 1.41888350e-01],
       [ 1.16194129e-01],
       [ 1.32630259e-01],
       [ 1.21245414e-01],
       [ 1.37395322e-01],
       [ 1.40814096e-01],
       [ 1.24776453e-01],
       [ 1.14690214e-01],
       [ 1.10452771e-01],
       [ 8.42811465e-02],
       [ 8.28155279e-02],
       [ 1.08722210e-01],
       [ 1.35615587e-01],
       [ 1.64932638e-01],
       [ 2.41662458e-01],
       [ 3.05614531e-01],
       [ 2.16573492e-01],
       [ 2.10804507e-01],
       [ 2.03640923e-01],
       [ 2.02726036e-01],
       [ 2.17061043e-01],
       [ 2.11467281e-01],
       [ 1.98435307e-01],
       [ 2.03826681e-01],
       [ 2.06593126e-01],
       [ 2.0

In [254]:
real_predictions = scaler.inverse_transform(y_predict)  # 0~1의 값으로 정규화된 값을 원래의 크기로 되돌린다.
# print("predicted next 5 min trade count", real_predictions[-1])  # 예측한 건수를 출력한다.

real_predictions



ValueError: non-broadcastable output operand with shape (715,1) doesn't match the broadcast shape (715,9)

In [170]:
x_test[14]

array([[4.72407300e+04, 4.72407600e+04, 4.71803800e+04, 5.37555000e+00,
        2.53763106e+05, 3.32000000e+02, 1.22749000e+00, 5.79562821e+04],
       [4.71959900e+04, 4.72365500e+04, 4.71927200e+04, 1.22461800e+01,
        5.78216273e+05, 3.84000000e+02, 5.33622000e+00, 2.51929215e+05],
       [4.72072000e+04, 4.72749800e+04, 4.71965100e+04, 1.51492000e+01,
        7.15603727e+05, 5.06000000e+02, 8.98866000e+00, 4.24579953e+05],
       [4.72595400e+04, 4.72734000e+04, 4.72500000e+04, 7.71686000e+00,
        3.64711302e+05, 3.60000000e+02, 3.39986000e+00, 1.60677519e+05],
       [4.72699900e+04, 4.73144700e+04, 4.72500000e+04, 1.22686600e+01,
        5.80187615e+05, 5.68000000e+02, 6.55229000e+00, 3.09865206e+05],
       [4.72663800e+04, 4.72999900e+04, 4.72500000e+04, 8.46017000e+00,
        3.99921145e+05, 3.73000000e+02, 2.36634000e+00, 1.11844237e+05],
       [4.72500100e+04, 4.72500200e+04, 4.71580100e+04, 1.89022100e+01,
        8.92377703e+05, 5.66000000e+02, 7.80930000e+00, 3.

In [145]:

model = tf.keras.models.load_model("resource/model/regression_20220415.h5")
# Calling `save('my_model.h5')` creates a h5 file `my_model.h5`.
model.save("regression_20220415.h5")

# It can be used to reconstruct the model identically.
# reconstructed_model = keras.models.load_model("my_h5_model.h5")

In [ ]:
import numpy as np
def create_dataset(signal_data, look_back=1):
    x_arr, y_arr = [], []
    for i in range(len(signal_data) - look_back):
        x_arr.append(signal_data[i:(i + look_back), 0])
        y_arr.append(signal_data[i + look_back, 0])

    x_arr = np.array(x_arr)
    x_arr = np.reshape(x_arr, (x_arr.shape[0], x_arr.shape[1], 1))
    return x_arr, np.array(y_arr)

klines_dataset = pd.read_csv('resource/dataset/binance/klines-BTCUSDT-1m_20220101090000_20220101205900.csv').to_numpy()

# 훈련
train = klines_dataset[0:int(len(klines_dataset) * 0.5)]
# 검증
val = klines_dataset[int(len(klines_dataset) * 0.5):int(len(klines_dataset) * 0.75)]
# 시험
test = klines_dataset[int(len(klines_dataset) * 0.75):-1]

x_train, y_train = create_dataset(train, look_back)
x_val, y_val = create_dataset(val, look_back)
x_test, y_test = create_dataset(test, look_back)
